# 最新版の日本語LLM

### はじめに
日本語のLLMは、オープンソースで公開されたLLMを日本語データセットで学習させることで公開されます。
現在はLLaMA 2（Metaによって開発されたLLM）が主な対象となって公開されています。これらのLLMは、ChatGPTの代替となるLLMがオープンソースで公開される流れをとっています。ChatGPT 3.5と比較する形でLLaMA2が公開されました。
ChatGPTには4.0があります。それに対応するLLMは、MiteriaというLLMがあります。これの日本語版のLLMも次期に公開されるでしょう。

### Swallowの概要
Swallow は、Llama 2の日本語能力を強化した大規模言語モデルで、パラメータサイズに応じて 7B、13B、70B の三種類があります。このモデルは、東京工業大学の岡崎研究室と横田研究室、国立研究開発法人産業技術総合研究所が共同で開発しました。モデルの特徴は、英語に加えて日本語の理解・生成能力に特化しており、継続的な事前学習によって日本語データの扱いを改善しています。

### 主な特徴
- 継続事前学習: Llama 2モデルに日本語の語彙を追加し、新たに開発した日本語データで継続的に学習を行っています。
- 高性能: 2023年12月時点で、日本語における最高性能を誇る大規模言語モデルです。
- 利用の自由度: LLAMA 2 Community Licenseに従い、研究や商業目的で自由に利用可能です。

In [ ]:
!pip install torch transformers sentencepiece accelerate protobuf tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 24.0 MB/s eta 0:00:00


In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "tokyotech-llm/Swallow-7b-instruct-hf"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, low_cpu_mem_usage=True, device_map="auto")


PROMPT_DICT = {
    "prompt_input": (
        "以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。"
        "リクエストを適切に完了するための回答を記述してください。\n\n"
        "### 指示:\n{instruction}\n\n### 入力:\n{input}\n\n### 応答:"

    ),
    "prompt_no_input": (
        "以下に、あるタスクを説明する指示があります。"
        "リクエストを適切に完了するための回答を記述してください。\n\n"
        "### 指示:\n{instruction}\n\n### 応答:"
    ),
}

def create_prompt(instruction, input=None):
    """
    Generates a prompt based on the given instruction and an optional input.
    If input is provided, it uses the 'prompt_input' template from PROMPT_DICT.
    If no input is provided, it uses the 'prompt_no_input' template.

    Args:
        instruction (str): The instruction describing the task.
        input (str, optional): Additional input providing context for the task. Default is None.

    Returns:
        str: The generated prompt.
    """
    if input:
        # Use the 'prompt_input' template when additional input is provided
        return PROMPT_DICT["prompt_input"].format(instruction=instruction, input=input)
    else:
        # Use the 'prompt_no_input' template when no additional input is provided
        return PROMPT_DICT["prompt_no_input"].format(instruction=instruction)

In [ ]:
# Example usage
instruction_example = "以下のトピックに関する詳細な情報をリストアップして、提案とその妥当性を示します。"
input_example = "福岡のご飯が美味しいと評判ですが、実際に観光地としてどういった魅力があるでしょうか。どんな観光体験が福岡に必要でしょうか"
prompt = create_prompt(instruction_example, input_example)

input_ids = tokenizer.encode(
    prompt,
    add_special_tokens=False,
    return_tensors="pt"
)

tokens = model.generate(
    input_ids.to(device=model.device),
    max_new_tokens=256,
    temperature=0.99,
    top_p=0.95,
    do_sample=True,
)

out = tokenizer.decode(tokens[0], skip_special_tokens=True)
print(out.replace("。", "。\n"))

### Nekomataの概要
rinnaでは、日本語の処理に適したGPT・BERT・HuBERT・CLIP・Stable Diffusionなどのテキスト・音声・画像に関する事前学習済み基盤モデルを公開することで、日本語のAI開発をサポートしてきました。2021年4月からrinnaが公開してきたモデルのダウンロード数は累計450万を超え、多くの研究・開発者にご利用いただいています。2023年10月には、英語を主なターゲットとし優れたパフォーマンスを誇るMeta社のLlama2に、日本語継続事前学習をすることで日本語においても高いパフォーマンスを実現した「Youri」シリーズを開発・公開しています。しかしYouriシリーズは、Llama2の英語を主なターゲットとする語彙サイズ3.2万のトークナイザーを利用しており、日本語を書き表すための語彙が不足しており推論効率が悪いという欠点がありました。Llama2以降には、英語だけでなく多言語をターゲットとした語彙サイズが大きいLLMが次々と公開されました。アリババ社が公開したQwenシリーズは語彙サイズが15.2万であり、日本語の推論においても高い効率を実現しています。

そこでrinnaは、Qwen-7Bと14Bに日本語の学習データを用いて継続事前学習を行い、高い日本語テキストの生成性能と推論速度を兼ね備えた「Nekomata」シリーズを開発し公開しました。さらに、汎用言語モデルに対話形式でユーザーの指示に応答するように追加学習した指示応答言語モデルも開発し、合計4モデルを公開しました。これらのモデル公開により、日本のAI研究・開発の更なる発展につながることを願っています。

・「Nekomata」シリーズのモデル一覧

https://huggingface.co/collections/rinna/nekomata-6582b5134ee85531becbb9a9

### 主な特徴
Nekomata 7Bと14Bは、70億パラメータのQwen-7B ( https://huggingface.co/Qwen/Qwen-7B ) と140億パラメータのQwen-14B ( https://huggingface.co/Qwen/Qwen-14B ) に対して、日本語と英語の学習データを用いてそれぞれ300億と660億トークンで継続事前学習したモデルです。Qwenの優れたパフォーマンスを日本語に引き継いでおり、日本語のタスクにおいて高い性能を示します。日本語言語モデルの性能を評価するためのベンチマークの一つである Stability-AI/lm-evaluation-harnessの9タスク平均スコアはNekomata 7Bが58.69、Nekomata 14Bが67.38なっています（図1）。また、日本語テキスト1byteに対するトークン数はLlama2/Youriが0.40、Qwen/Nekomataが0.24であり、推論効率が高くなっています。モデル名の由来は、妖怪の「猫又（ねこまた）」からきています。



In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("rinna/nekomata-14b-instruction", trust_remote_code=True)

# Use GPU with bf16
# model = AutoModelForCausalLM.from_pretrained("rinna/nekomata-14b-instruction", device_map="auto", trust_remote_code=True, bf16=True)

# Use GPU with fp16
# model = AutoModelForCausalLM.from_pretrained("rinna/nekomata-14b-instruction", device_map="auto", trust_remote_code=True, fp16=True)

# Use CPU
# model = AutoModelForCausalLM.from_pretrained("rinna/nekomata-14b-instruction", device_map="cpu", trust_remote_code=True)

# Automatically select device and precision
model = AutoModelForCausalLM.from_pretrained("rinna/nekomata-14b-instruction", device_map="auto", trust_remote_code=True)

instruction_example = "以下のトピックに関する詳細な情報をリストアップして、提案とその妥当性を示します。"
input_example = "福岡のご飯が美味しいと評判ですが、実際に観光地としてどういった魅力があるでしょうか。どんな観光体験が福岡に必要でしょうか"

prompt = f"""
以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示:
{instruction_example}

### 入力:
{input_example}

### 応答:
"""
token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=200,
        do_sample=True,
        temperature=0.5,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

out = tokenizer.decode(output_ids.tolist()[0])
print(out.replace("。", "。\n"))